In [ ]:
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# DB 연결
from DB import *
db_connection = connection_db()

In [ ]:
# df_eco : 생태통로
df_eco = load_data_from_rds('생태통로', db_connection)
df_eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      497 non-null    int64  
 1   위도      497 non-null    float64
 2   경도      497 non-null    float64
dtypes: float64(2), int64(1)
memory usage: 11.8 KB


In [ ]:
# df1 : 국립생태원 PDF 규격 데이터
df1 = pd.read_csv('.././data/variables/생태통로_국립생태원PDF.csv')
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   위도      66 non-null     float64
 1   경도      66 non-null     float64
 2   연장      66 non-null     float64
 3   폭       65 non-null     float64
 4   높이      20 non-null     float64
 5   개방도     20 non-null     float64
dtypes: float64(6)
memory usage: 3.2 KB


In [ ]:
# df2 : 환경부 PDF 규격 데이터
df2 = pd.read_csv('.././data/variables/생태통로_환경부PDF.csv')
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152 entries, 0 to 151
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   위도      150 non-null    float64
 1   경도      150 non-null    float64
 2   연장      152 non-null    float64
 3   폭       152 non-null    float64
 4   높이      30 non-null     float64
 5   개방도     30 non-null     float64
dtypes: float64(6)
memory usage: 7.2 KB


In [ ]:
# df_eco와 df를 좌표 기준으로 병합
df1 = pd.merge(df_eco, df1, how='outer', on=['위도','경도'])
df2 = pd.merge(df_eco, df2, how='outer', on=['위도','경도'])

In [ ]:
# df_eco에 df1의 규격 데이터 추가
col_list = ['폭', '연장', '높이', '개방도']
df_eco = pd.concat([df_eco, df1[col_list]], axis=1)
df_eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      497 non-null    int64  
 1   위도      497 non-null    float64
 2   경도      497 non-null    float64
 3   폭       65 non-null     float64
 4   연장      66 non-null     float64
 5   높이      20 non-null     float64
 6   개방도     20 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 27.3 KB


In [ ]:
# df_eco에 df2의 규격 데이터 추가
idx = list(df_eco[((df_eco[col_list[0]].isnull()) | df_eco[col_list[1]].isnull())].index)
for i in idx:
  for col in col_list:
    df_eco.loc[i, col] = df2.loc[i, col]
df_eco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   번호      497 non-null    int64  
 1   위도      497 non-null    float64
 2   경도      497 non-null    float64
 3   폭       187 non-null    float64
 4   연장      187 non-null    float64
 5   높이      43 non-null     float64
 6   개방도     43 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 27.3 KB


In [ ]:
# DF to RDS
upload_data_to_rds(df_eco, '생태통로', db_connection)